In [26]:
from girder_client import GirderClient
import pandas as pd
import birdshot2

# Authenticate with GirderClient
client = GirderClient(apiUrl="https://data.htmdec.org/api/v1")
client.authenticate(apiKey="MFfpVN81hmOaUV7cTGsovnzdr0iB87ygR0RxkDYA")

# List of campaigns to analyze
campaigns =  ["AAA", "AAB", "AAC", "AAD", "AAE", "BAA", "BBA", "BBB", "BBC", "CBA"]

# Initialize DataFrames to store results
all_samples_missing = []
all_columns_missing = []

# Loop through each campaign and calculate missing values
for campaign in campaigns:
    # Query the data for the given campaign
    df = birdshot2.query(campaign=campaign, client=client)
    
    # Count the number of missing values for each sample (rows)
    row_missing = df.isna().sum(axis=1)
    row_missing.name = "Missing Values"
    row_missing = row_missing.reset_index()
    row_missing.columns = ['Sample', 'Missing Values']
    row_missing['Campaign'] = campaign  # Add campaign identifier for grouping
    all_samples_missing.append(row_missing)
    
    # Count the number of missing values for each column
    column_missing = df.isna().sum(axis=0)
    column_missing.name = campaign
    all_columns_missing.append(column_missing)

# Combine results for all samples into a single DataFrame
samples_missing_df = pd.concat(all_samples_missing, ignore_index=True)

# Add a row for the sum of missing values by campaign
campaign_sums = samples_missing_df.groupby('Campaign')['Missing Values'].sum().reset_index()
campaign_sums['Sample'] = campaign_sums['Campaign']  # Use campaign name as the sample name
samples_missing_df = pd.concat([samples_missing_df, campaign_sums], ignore_index=True)

# Combine results for all columns into a single DataFrame
columns_missing_df = pd.concat(all_columns_missing, axis=1).T  # Transpose to make campaigns rows

# Add a row for the total missing values across all campaigns
columns_missing_df.loc['Total'] = columns_missing_df.sum(axis=0)

# Add a column for the sum of missing values across rows for each campaign
columns_missing_df['Total'] = columns_missing_df.sum(axis=1)

# Sort columns alphabetically (excluding the 'Row Sum' column, which stays at the end)
columns_to_sort = [col for col in columns_missing_df.columns if col != 'Total']
columns_missing_df = columns_missing_df[sorted(columns_to_sort) + ['Total']]

# Save the results to CSV files
samples_missing_df.to_csv('per_sample.csv', index=False)
columns_missing_df.to_csv('per_column.csv', index=True)